# Bibliotecas

In [23]:
import numpy as np
import pandas as pd
import geopy.distance as gp

# Carga dos Dados

In [24]:
poi = pd.read_csv('../data/base_pois_def.csv')
position = pd.read_csv('../data/posicoes.csv')

In [25]:
# Converte data_posicao para datetime
position['data_posicao'] = pd.to_datetime(position['data_posicao'], format="%a %b %d %Y %H:%M:%S GMT%z", exact=False)

# Determina se o veiculo estah parado (on nao) em cada registro
position['parado'] = position.apply(lambda x: (not x.ignicao and x.velocidade < 5), axis = 1)

# Calcula o tempo(s) entre dois registros consecutivos
position = position.sort_values(by=['placa','data_posicao'])
position['tempo'] = (position['data_posicao'] - position.groupby(['placa'])['data_posicao'].shift(1)).dt.seconds

# 'JOIN' dos dois DataFrames

In [26]:
data = []
for aPos in position.itertuples():
    for aPoi in poi.itertuples():
        # Calcula a distancia do veiculo ao POI
        dist = gp.distance((aPos.latitude, aPos.longitude), (aPoi.latitude, aPoi.longitude)).meters
        # Cria um registro, se o veiculo estiver em um POI
        if (dist <= aPoi.raio):
            data.append({'placa': aPos.placa, 'tempo': aPos.tempo, 'poi': aPoi.nome, 'parado': aPos.parado})

positionPoi = pd.DataFrame(data)

# Consolidacao dos tempos

In [27]:
result = positionPoi.groupby(['placa','poi','parado'], as_index=False)['tempo'].sum()
result

,placa,poi,parado,tempo
0,CAR0012,PONTO 24,False,1604.0
1,CAR0012,PONTO 24,True,342764.0
2,TESTE001,PONTO 1,False,3876.0
3,TESTE001,PONTO 1,True,471.0
4,TESTE001,PONTO 2,False,6001.0
5,TESTE001,PONTO 2,True,2432.0
6,TESTE001,PONTO 24,False,6408.0
7,TESTE001,PONTO 24,True,653235.0


In [28]:
# Quantidade de tempo que os veiculos passaram parados dentro de cada POI
carStoppedPoi = result[result['parado']==True].groupby(['placa','poi'], as_index=False)['tempo'].sum()
carStoppedPoi

,placa,poi,tempo
0,CAR0012,PONTO 24,342764.0
1,TESTE001,PONTO 1,471.0
2,TESTE001,PONTO 2,2432.0
3,TESTE001,PONTO 24,653235.0


In [29]:
# Quantidade de tempo que os veiculos estavam dentro de cada POI
carOnPoi = result.groupby(['placa','poi'], as_index=False)['tempo'].sum()
carOnPoi

,placa,poi,tempo
0,CAR0012,PONTO 24,344368.0
1,TESTE001,PONTO 1,4347.0
2,TESTE001,PONTO 2,8433.0
3,TESTE001,PONTO 24,659643.0


In [30]:
# Tempo total da frota gasto parado em cada POI
fleetStoppedPoi = result[result['parado']==True].groupby(['poi'], as_index=False)['tempo'].sum()
fleetStoppedPoi

,poi,tempo
0,PONTO 1,471.0
1,PONTO 2,2432.0
2,PONTO 24,995999.0


In [31]:
# Tempo total parado por veiculo, independente do POI
carStopped = position[position['parado']==True].groupby(['placa'], as_index=False)['tempo'].sum()
carStopped

,placa,tempo
0,CAR0012,342764.0
1,TESTE001,655750.0


# Consolidacao dos dados

In [62]:
data = []
greatTotal = 0
car = pd.unique(carStopped['placa'])
for aCar in car:
    st = {'placa': aCar}
    # Adiciona colunas com o tempo parado em cada POI
    for aRec in carStoppedPoi[carStoppedPoi['placa'] == aCar].itertuples():
        st[aRec.poi.replace(' ','_').lower()+'_parado'] = aRec.tempo

    # Adiciona colunas com o tempo em cada POI
    for aRec in carOnPoi[carOnPoi['placa'] == aCar].itertuples():
        st[aRec.poi.replace(' ','_').lower()+'_total'] = aRec.tempo
    
    # Adiciona coluna com o tempo total parado
    for aRec in carStopped[carStopped['placa'] == aCar].itertuples():
        st['total_parado'] = aRec.tempo
        greatTotal += aRec.tempo
    
    data.append(st)

st = {'placa': 'FROTA'}
# Adiciona colunas com o tempo da frota parada em cada POI
for aRec in fleetStoppedPoi.itertuples():
    st[aRec.poi.replace(' ','_').lower()+'_parado'] = aRec.tempo
st['total_parado'] = greatTotal
data.append(st)

output = pd.DataFrame(data).fillna(0)
output

,placa,ponto_24_parado,ponto_24_total,total_parado,ponto_1_parado,ponto_2_parado,ponto_1_total,ponto_2_total
0,CAR0012,342764.0,344368.0,342764.0,0.0,0.0,0.0,0.0
1,TESTE001,653235.0,659643.0,655750.0,471.0,2432.0,4347.0,8433.0
2,FROTA,995999.0,0.0,998514.0,471.0,2432.0,0.0,0.0


# Output dos resultados

In [63]:
cols = output.columns.tolist()
cols.sort()
output[cols].to_csv('../output/resultados_consolidados_POIs.csv', index=False)